# Kaixin's 23Na(3He,d) data
See https://pfunk.readthedocs.io/en/latest/guide/bays_fit.html

### Import Packages

In [18]:
%matplotlib inline
import pfunk
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from num2tex import num2tex
from num2tex import configure as num2tex_configure
import corner
import pickle
import numpy as np
from scipy import stats
from scipy.signal import argrelmax
import copy
import os
import pandas as pd
from datetime import datetime
from time import sleep
from tqdm import tqdm
from KaixinSpecTools import * # Some code I wrote to calculate theoretical optical model parameters from model 
from scipy.stats import uniform # This is missing in pfunk and caused a bug
np.set_printoptions(suppress=True)
num2tex_configure(display_singleton=1)
num2tex_configure(exp_format='times')
path_main='/home/kaixin/Documents/23NaHe3d_Analysis' # main directory
path_fresco_input=path_main+'/FrescoInputFiles' # directory for fresco input files
path_fresco_data=path_main+'/FrescoInputData' # directory for yield data used for fitting optical model parameters in fresco input.
# path_mcmc=path_main+'/20221211_quick'
path_mcmc=path_main+'/20240626_complete' #directory to store MCMC picle files and results
path_fresco_temp=path_main+'/FrescoTemp' # directory to run fresco and create temporary output files
path_log=path_main+'/RunLog/'# directory to save information of each run
path_plots=path_main+'/Plots/'

num2tex configure options:
 - help_text: boolean whether this help text is displayed when configure() is called
 - exp_format: string format specification for how the power of 10 is displayed in scientific format.
     - also accepts specifiers 'times', which is converted to '\times 10^{}', 'cdot': '\cdot 10^{}', and 'parentheses': '(10^{})'
 - display_singleton: boolean on whether the "1 \times" is printed in "1 \times 10^{p}"
          
num2tex configure options:
 - help_text: boolean whether this help text is displayed when configure() is called
 - exp_format: string format specification for how the power of 10 is displayed in scientific format.
     - also accepts specifiers 'times', which is converted to '\times 10^{}', 'cdot': '\cdot 10^{}', and 'parentheses': '(10^{})'
 - display_singleton: boolean on whether the "1 \times" is printed in "1 \times 10^{p}"
          


In [19]:
# level_list=['7349_twol','7555','7747_twol','8358','8437_twol','8654_twol','8864','9003','9146_1p12','9146','9301','9458','9516','9828','9828_twol',\
#             '9965','9965_twol','10028','10059_twol','10161','10333_1p32','10361','10660','10712','10712_twol','10724','10724_twol','10731','10731_twol','10821',\
#             '10821_twol','10917','11217','11314','11390_1p12','11390',\
#             '11453_twol','11518_twol','11695','11933']

level_list=['10712','10731']
# level_list=['9301','9301_twol','9828_twol','10731','10731_twol','10712','10712_twol','8864_1p12','11217']
Notes="Make complete MCMC pickle files in a batch."

ambiguity=True # To resolve ambiguities according to Caleb Thesis section 6.9.2
# c0_restrain=129 # from one of my run. No big difference from Caleb's.
c0_restrain=132.9
Init_Fit=False
run_MCMC=False 
now = datetime.now()
current_time = now.strftime("%Y%m%d_%H:%M")
os.chdir(path_fresco_temp)
if (os.path.exists(current_time)==False):
    os.mkdir(current_time)
os.chdir(current_time)

for level in level_list:
    if (level.find('_twol')>=0):
        method='_twol'
        # ambiguity=False # ?
        fresco_path = path_fresco_input+'/Na23He3d_'+level[0:level.find('_twol')]+'keV_twol'+'.in'
    elif (level.find('1p')>=0):
        method='_'+level[level.find('1p'):level.find('1p')+4]
        fresco_path = path_fresco_input+'/Na23He3d_'+level[0:level.find('1p')-1]+'keV'+method+'.in'
#         level=level[0:level.find('1p')-1]
    else:
        method=''
        fresco_path = path_fresco_input+'/Na23He3d_'+level+'keV'+method+'.in'

    fresco_names = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6',
                    'p1', 'p2', 'p3', 'p4', ('p5','p5'), ('p6','p6'),'p4']
    fresco_positions = [81, 82, 83, 89, 90, 91,
                        111, 112, 113, 114, (115,123), (116,124), 122]
    elastic_data_path = path_fresco_data+'/23Na-3He-ES.data'
    if (method==''):  
        transfer_data_path = path_fresco_data+'/23Na-3He-d-'+level+'keV.data'
    elif (method=='_twol'):
        transfer_data_path = path_fresco_data+'/23Na-3He-d-'+level[0:level.find('_twol')]+'keV.data'
    else:
        transfer_data_path = path_fresco_data+'/23Na-3He-d-'+level[0:level.find('_1p')]+'keV.data'
    model = pfunk.model.Model(fresco_path, fresco_names, fresco_positions)
    model.create_norm_prior([-10], [10.0])
#     percents = np.array([.2, .2, .2, .2, .2, .2,
#                      .1, .1, .1, .1, .1, .1, .1])
#     model.create_pot_prior(model.fresco.x0, model.fresco.x0*percents)  # Use original params in fresco input files
    init_3He=np.array(OptPotCal.Liang3He(Ep=21,Nt=12,Zt=11,At=23)) # To calculate from model, Otherwise should be taken from
    if (method==''):  
        init_2H=np.array(OptPotCal.Daehnick2H(Ex=float(level)/1000,qval=6.1992,Nt=12,Zt=12,At=24)) # Ex is in MeV
    elif method=='_twol':  
        init_2H=np.array(OptPotCal.Daehnick2H(Ex=float(level[0:-5])/1000,qval=6.1992,Nt=12,Zt=12,At=24)) # Ex is in MeV
        init_values=np.concatenate((init_3He,init_2H))
    else:
        init_2H=np.array(OptPotCal.Daehnick2H(Ex=float(level[0:level.find('1p')-1])/1000,qval=6.1992,Nt=12,Zt=12,At=24)) # Ex is in MeV
    init_values=np.concatenate((init_3He,init_2H))
    percents=np.array([.2]*len(init_3He)+[.1]*len(init_2H))
    model.create_pot_prior(init_values, init_values*percents)

    model.create_scatter_prior(widths=[0.1])  ## width -> 0.01 # Scatter data scale factor
    model.create_scatter_prior()  ## width -> 0.01 # Transfer data scale factor
    model.create_spec_prior([0.0], [1.5])  # C2S, position 1  #1.5 ->0.2 for 8358
    model.create_spec_prior([1.0], [0.15], gaus=True) # D0, position 2
    if (method=='_twol' ):
        model.create_spec_prior(0.5,0.4,mixing_percent=True) # The mixing parameter, alpha. 

    model.create_prior()
    iNorm = 0
    iC2S  = 1
    iD0=2
    iScat = 3
    iTrans = 4
    iV=5 ## Index of first optical parameter
    # iV = 4
    model.x0[iNorm] = -1   ## Norm factor
    model.x0[iD0] = 1 #D0
    model.x0[iC2S]  = 0.08  ## C2S
    model.x0[iScat] = 0.1   ## Scat
    model.x0[iTrans] = 0.1   ## Trans

    if (method=='_twol'):
        iAlpha=iV
        iV=iV+1
        model.x0[iAlpha]=0.5
    model.create_elastic_likelihood('fort.201', elastic_data_path, norm_index=iNorm,scatter_index=iScat)
    if (method=='_twol'):
        model.create_two_l_transfer_likelihood('fort.202', 'fort.203',transfer_data_path, sf_index=[iC2S,iD0], percent_index=iAlpha, norm_index=iNorm,scatter_index=iTrans)
    else:
        model.create_transfer_likelihood('fort.202', transfer_data_path, [iC2S,iD0], norm_index=iNorm,scatter_index=iTrans)
    #     model.create_transfer_likelihood('fort.202', transfer_data_path, [iC2S,iD0], norm_index=iNorm,scatter_index=None)
    if (ambiguity):
            model.create_vr_likelihood(iV, iV+1, c0_restrain, percent=0.20, n=1.4)
    model.create_likelihood()
    model.run_fresco(model.x0)
    d = model.likelihood[1].data
    if (method=='_twol'):
        cs_orig1 = pfunk.fresco_classes.read_cross('./fort.202')
        cs_orig2 = pfunk.fresco_classes.read_cross('./fort.203')
        [scale,alpha]=Minimizations.AlphaMinimize(d,cs_orig1,cs_orig2,LogScale=False).x
    if (Init_Fit):
        fit = pfunk.model_fit.MAPFit(model, percent_range=1.0)
        fit.run_anneal(max_iter=100)
        print("\n finished")
#         print(fit.x0_custom)
        print(fit.results.x)
        model.x0=fit.results.x
    
    sampler = pfunk.sampler.Sampler(model)
    sampler.nwalker = 400   #500
    sampler.nstep = 8000 #1500
    if not os.path.exists(path_mcmc+'/'+level):
        os.makedirs(path_mcmc+'/'+level)
    f_myfile = open(path_mcmc+'/'+level+'/Model-'+level+'.pickle', 'wb')
    pickle.dump(model, f_myfile)
    f_myfile.close()
    os.popen('cp '+path_mcmc+'/runMCMC.py '+path_mcmc+'/'+level) 
    # os.popen('cp '+path_mcmc+'/fresco '+path_mcmc+'/'+level) 
    
    logdir=path_log+current_time+'_'+level+'_'+method
    if (os.path.exists(logdir)==False):
        os.mkdir(logdir)
    
    # fp = open(logdir+'/ModelParams.txt', 'w')
    # if (Init_Fit):
    #     fp.write("Initial fit.x0_custom values:\n")
    #     for value in fit.x0_custom:
    #         fp.write(str(value))
    #         fp.write('\t')
    #     fp.write('\n')    
    #     fp.write("Fitted fit.results.x values:\n")
    #     for value in fit.results.x:
    #         fp.write(str(value))
    #         fp.write('\t')
    #     print('Done')
    # else:
    #     fp.write("Initial fit.x0_custom values:\n")
    #     for value in model.x0:
    #         fp.write(str(value))
    #         fp.write('\t')
    #     fp.write('\n')    
    # fp.close()

    fp = open(logdir+'/Notes.txt', 'w')
    fp.write(Notes)
    fp.close()
    if (Init_Fit):
        fp = open(path_log+'Fits.txt', 'a')
        fp.write(level)
        fp.write('\t')
        fp.write(str(fit.results.x[iC2S]))
        fp.write('\n')
        fp.close()
    fp = open(path_log+'/Log.txt', 'a')
    fp.write(current_time)
    fp.write('\t')
    fp.write(level)
    fp.write('\t')
    if (Init_Fit):
        fp.write(str(fit.results.x[iC2S]))
        fp.write('\t')
    fp.write(Notes)
    fp.write('\n')
    fp.close()

In [3]:
# break here

In [9]:
# Read files
path_mcmc='/media/kaixin/BACKUP3/Kaixin\'sFolder/MCMC_runs'
path_plots=path_mcmc+'/Plots'
level_list=[]
for filename in os.listdir(path_mcmc):
    right=filename.find('.npy')
    left=filename.find('samples-')
    if left>=0 and right>=0:
        level_list.append(filename[left+8:right])

In [11]:
Ref=pd.read_csv(path_mcmc+'/C2S_Results',sep='\t')
C2S_results=[]
ANC_results=[]
ambiguity=True
c0_restrain=132.9
skip=True
lcolor = ['#9BEFA5','#F5B82E','#C08497','#9E2B25','#B4E1FF']
# fC2S = open("C2S.txt",'w')
# for level in level_list:
for level in ['9003']:
    if level in ['9828_400_highc2s']:
        continue
    # if level in ['9003']:
    #     skip=False
    # elif skip:
    #     continue
    if level.find('_')>=0:
        levelnum=int(level[0:level.find('_')])
    else:
        levelnum=int(level)
    if (level.find('_twol')>=0):
        method='_twol'
        # ambiguity=False # ?
        fresco_path = path_fresco_input+'/Na23He3d_'+level[0:level.find('_twol')]+'keV_twol'+'.in'
    elif (level.find('1p')>=0):
        method='_'+level[level.find('1p'):level.find('1p')+4]
        fresco_path = path_fresco_input+'/Na23He3d_'+level[0:level.find('1p')-1]+'keV'+method+'.in'
    else:
        method=''
        fresco_path = path_fresco_input+'/Na23He3d_'+level+'keV'+method+'.in'
    fresco_names = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6',
                    'p1', 'p2', 'p3', 'p4', ('p5','p5'), ('p6','p6'),'p4']
    fresco_positions = [81, 82, 83, 89, 90, 91,
                        111, 112, 113, 114, (115,123), (116,124), 122]
    elastic_data_path = path_fresco_data+'/23Na-3He-ES.data'
    if (method==''):  
        transfer_data_path = path_fresco_data+'/23Na-3He-d-'+level+'keV.data'
    elif (method=='_twol'):
        transfer_data_path = path_fresco_data+'/23Na-3He-d-'+level[0:level.find('_twol')]+'keV.data'
    else:
        transfer_data_path = path_fresco_data+'/23Na-3He-d-'+level[0:level.find('_1p')]+'keV.data'
        print(transfer_data_path)
    model = pfunk.model.Model(fresco_path, fresco_names, fresco_positions)
    model.create_norm_prior([-10], [10.0])
#     percents = np.array([.2, .2, .2, .2, .2, .2,
#                      .1, .1, .1, .1, .1, .1, .1])
#     model.create_pot_prior(model.fresco.x0, model.fresco.x0*percents)  # Use original params in fresco input files
    init_3He=np.array(OptPotCal.Liang3He(Ep=21,Nt=12,Zt=11,At=23)) # To calculate from model, Otherwise should be taken from
    if (method==''):  
        init_2H=np.array(OptPotCal.Daehnick2H(Ex=float(level)/1000,qval=6.1992,Nt=12,Zt=12,At=24)) # Ex is in MeV
    elif method=='_twol':  
        init_2H=np.array(OptPotCal.Daehnick2H(Ex=float(level[0:level.find('_')])/1000,qval=6.1992,Nt=12,Zt=12,At=24)) # Ex is in MeV
        init_values=np.concatenate((init_3He,init_2H))
    else:
        init_2H=np.array(OptPotCal.Daehnick2H(Ex=float(level[0:level.find('1p')-1])/1000,qval=6.1992,Nt=12,Zt=12,At=24)) # Ex is in MeV
    init_values=np.concatenate((init_3He,init_2H))
    percents=np.array([.2]*len(init_3He)+[.1]*len(init_2H))
    model.create_pot_prior(init_values, init_values*percents)

    model.create_scatter_prior(widths=[0.1])  ## width -> 0.01 # Scatter data scale factor
    model.create_scatter_prior()  ## width -> 0.01 # Transfer data scale factor
    model.create_spec_prior([0.0], [1.5])  # C2S, position 1  #1.5 ->0.2 for 8358
    model.create_spec_prior([1.0], [0.15], gaus=True) # D0, position 2
    if (method=='_twol' ):
        model.create_spec_prior(0.5,0.4,mixing_percent=True) # The mixing parameter, alpha. 

    model.create_prior()
    iNorm = 0
    iC2S  = 1
    iD0=2
    iScat = 3
    iTrans = 4
    iV=5 ## Index of first optical parameter
    # iV = 4
    model.x0[iNorm] = -1   ## Norm factor
    model.x0[iD0] = 1 #D0
    model.x0[iC2S]  = 0.08  ## C2S
    model.x0[iScat] = 0.1   ## Scat
    model.x0[iTrans] = 0.1   ## Trans

    if (method=='_twol'):
        iAlpha=iV
        iV=iV+1
        model.x0[iAlpha]=0.5
    model.create_elastic_likelihood('fort.201', elastic_data_path, norm_index=iNorm,scatter_index=iScat)
    if (method=='_twol'):
        model.create_two_l_transfer_likelihood('fort.202', 'fort.203',transfer_data_path, sf_index=[iC2S,iD0], percent_index=iAlpha, norm_index=iNorm,scatter_index=iTrans)
    else:
        model.create_transfer_likelihood('fort.202', transfer_data_path, [iC2S,iD0], norm_index=iNorm,scatter_index=iTrans)
    #     model.create_transfer_likelihood('fort.202', transfer_data_path, [iC2S,iD0], norm_index=iNorm,scatter_index=None)
    if (ambiguity):
            model.create_vr_likelihood(iV, iV+1, c0_restrain, percent=0.20, n=1.4)
    model.create_likelihood()


    
    samples = np.load(path_mcmc+'/samples-'+level+'.npy')
    print('Loaded level - '+level)
    Nchains=int(len(samples[1,:,1]))
    Nstep=(len(samples[:,1,1]))
    Ndiscard = int(Nstep*3/4) 
    Nthin = 20
    sliced_samples = samples[Ndiscard:,:,:]
    thinned_samples = sliced_samples[int(Nthin/2)::Nthin,:]
    flattened_samples = thinned_samples.reshape(-1,sliced_samples.shape[-1])
    s = flattened_samples
    snew=[]
    upperlim=np.average(np.percentile(s[:, 1], [16, 50, 84]))+\
        4*np.diff(np.percentile(s[:, 1], [16, 50, 84]))[1]
    for i in range(0,len(s)):
        Filter=s[i,iC2S]<upperlim and s[i,iC2S]>0  and s[i,iV]*s[i,iV+1]**1.14 >100 and s[i,iV]*s[i,iV+1]**1.14 <150
        if(Filter):
            snew.append(s[i,:])
    print('Discarded diverged chains: %d/%d' % (len(s)-len(snew),len(s)))
    s=np.array(snew)

    # Run fresco and plot
    N = 1000
    s_samp = s[np.random.choice(s.shape[0],N),:]
    all_cs_elastic = []
    all_cs_transfer = []
    all_cs_transfer_l2 = []
    for i in tqdm(range(N)):
        ele=s_samp[i]
        model.run_fresco(ele)
        cs_elastic_temp = pfunk.fresco_classes.read_cross('fort.201')
        cs_transfer_temp = pfunk.fresco_classes.read_cross('fort.202')
        if ('twol' in method):
            cs_transfer_temp2 = pfunk.fresco_classes.read_cross('fort.203')
            all_cs_transfer_l2.append(cs_transfer_temp2)
        all_cs_elastic.append(cs_elastic_temp)
        all_cs_transfer.append(cs_transfer_temp)
        
    data=pd.read_csv('fort.46',index_col=False,lineterminator='\n',sep='\s+',header=None,names=['R','ANC','Unknown1','Unknown2','Unknown3','Unknown4'])

    if ('twol' in method):
        ANC_results.append(level+'\t'+str(data['ANC'][0]))
        ANC_results.append(level+'\t'+str(data['ANC'][1]))
        s1=np.percentile(s[:, iC2S]*s[:, iAlpha], [16, 50, 84])
        s2=np.percentile(s[:, iC2S]*(1-s[:, iAlpha]), [16, 50, 84])
        txt = "{3} = ${0:.4f}_{{{1:.4f}}}^{{{2:.4f}}}$"
        txt=txt.format(2/3*s1[1],2/3*(-s1[1]+s1[0]),2/3*(s1[2]-s1[1]), '${C^2S}_{'+level+'}$')
        C2S_results.append(level+'\t'+txt)
        txt = "{3} = ${0:.4f}_{{{1:.4f}}}^{{{2:.4f}}}$"
        txt=txt.format(2/3*s2[1],2/3*(-s2[1]+s2[0]),2/3*(s2[2]-s2[1]), '${C^2S}_{'+level+'}$')
        C2S_results.append(level+'\t'+txt)
    else:
        ANC_results.append(level+'\t'+str(data['ANC'][0]))
        mcmc = np.percentile(s[:, iC2S], [16, 50, 84])
        q = np.diff(mcmc)
        txt = "{3} = ${0:.4f}_{{-{1:.4f}}}^{{{2:.4f}}}$"
        txt = txt.format(mcmc[1]*2/3, q[0]*2/3, q[1]*2/3, '${C^2S}_{'+level+'}$')
        C2S_results.append(level+'\t'+txt)
    all_cs_elastic_Norm = copy.deepcopy(all_cs_elastic)
    all_cs_transfer_Norm = copy.deepcopy(all_cs_transfer)
    all_cs_transfer_Norm_l1 = copy.deepcopy(all_cs_transfer)
    all_cs_transfer_Norm_l2 = copy.deepcopy(all_cs_transfer_l2)
    for i in range(N):
        all_cs_elastic_Norm[i].sigma = 10**(s_samp[i,iNorm])*all_cs_elastic[i].sigma
        if ('twol' in method):
    #         alpha=0.7737467799020796
            alpha=np.percentile(s[:, iAlpha], [16, 50, 84])[2]
    #         alpha=np.percentile(s[:, iAlpha], [84])[0]
            all_cs_transfer_Norm[i].sigma = (1-alpha)*10**(s_samp[i,iNorm])*s_samp[i,iC2S]*all_cs_transfer_l2[i].sigma \
            + (alpha)*10**(s_samp[i,iNorm])*s_samp[i,iC2S]*all_cs_transfer[i].sigma
            all_cs_transfer_Norm_l1[i].sigma = (alpha)*10**(s_samp[i,iNorm])*s_samp[i,iC2S]*all_cs_transfer[i].sigma        
            all_cs_transfer_Norm_l2[i].sigma = (1-alpha)*10**(s_samp[i,iNorm])*s_samp[i,iC2S]*all_cs_transfer_l2[i].sigma
    #         all_cs_transfer_Norm[i].sigma = (1-alpha)*10**(s_samp[i,iNorm])*s_samp[i,iC2S]*all_cs_transfer_l2[i].sigma \
    #         + (alpha)*10**(s_samp[i,iNorm])*s_samp[i,iC2S]*all_cs_transfer[i].sigma
    #         all_cs_transfer_Norm_l1[i].sigma = (alpha)*10**(s_samp[i,iNorm])*s_samp[i,iC2S]*all_cs_transfer[i].sigma        
    #         all_cs_transfer_Norm_l2[i].sigma = (1-alpha)*10**(s_samp[i,iNorm])*s_samp[i,iC2S]*all_cs_transfer_l2[i].sigma
        else:
            all_cs_transfer_Norm[i].sigma = 10**(s_samp[i,iNorm])*s_samp[i,iC2S]*all_cs_transfer[i].sigma
            
    d = model.likelihood[0].data
    plt.figure(figsize=(9.5, 6.5))
    pfunk.utilities.plot_ci(all_cs_elastic_Norm,colors=['#AB87FF','#a6cee3'])
    plt.ylim(3e-4, 1)
    plt.xlim(1e-4, 75)
    ymin=min(d.sigma)/4
    ymax=max(d.sigma)*2.5
    plt.ylim(ymin,ymax)
    plt.ylabel(r'$Yield(arb.units)$')
    plt.xlabel(r'$\theta_{c.\!m.}$ (deg)')
    # plt.title(level, fontsize=16)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    # plt.yticks([5e-2,1e-1,2e-1,4e-1],['$'+'{:.0e}'.format(num2tex(i))+'$'  for i in [5e-2,1e-1,2e-1,4e-1]],fontsize=20)
    # ax.yaxis.set_major_formatter(LogFormatterMathtext())
    plt.minorticks_on()
    plt.tight_layout()
    plt.grid(which='major', axis='both',linestyle='--')
    if(True): ##plot global curve with legend
        model.run_fresco(model.x0)
        cs = pfunk.fresco_classes.read_cross('fort.201')
        glob, =plt.plot(cs.theta, cs.sigma*10**model.x0[0], label='Global Optical Model', lw=1.5,color='red')
        data_label, =plt.plot(d.theta, d.sigma, linestyle="None",marker='o',zorder=1, color="#ff7f00",label='experiment')
        pop_a = mpatches.Patch(color='#AB87FF', label='68%')
        pop_b = mpatches.Patch(color='#a6cee3', label='95%')
        plt.legend(handles=[pop_a,pop_b,glob,data_label],fontsize=22)
        plt.ylabel(r'$\sigma(arb.units)$')
    # plt.savefig(path_plots+'/'+level+'_ES.png')

    d = model.likelihood[1].data
    # Read J and orbital to be updated to the figure
    orbital=Ref.loc[Ref['Level']==levelnum]['Orbital'].values
    jf=Ref.loc[Ref['Level']==levelnum]['Jf'].values[0]
    if 'twol' in method and len(orbital)>2:
        orbital=orbital[1:3]
    ls=[]
    for s in orbital:
        if s.find('2s')>=0:
            ls.append(0)
        if s.find('p')>=0:
            ls.append(1)
        if s.find('1d')>=0:
            ls.append(2)
        if s.find('1f')>=0:
            ls.append(3)
        if s.find('1g')>=0:
            ls.append(4)
    # plt.figure(figsize=(9.5, 6.5))
    if level=='7747_twol':
        level='7748_twol'
    elif level=='8654_twol':
        level='8655_twol'
    elif level=='10333_1p32':
        level='10334_1p32'
    elif level=='11217':
        level='11207'
    elif level=='11518_twol':
        level='11522_twol'
    elif level=='11695':
        level='11698'
    fig, ax = plt.subplots(figsize=(9.5, 6.5))
    plt.yscale('log')
    if ('twol' in method):
        
        pfunk.utilities.plot_ci(all_cs_transfer_Norm,colors=['#AB87FF'], data=d,zorder=0,linewidth=4.0,markersize=10.0,levels=[68.0])
        pfunk.utilities.plot_ci(all_cs_transfer_Norm_l1,colors=[lcolor[ls[0]]], alpha=0.7,zorder=1,levels=[68.0])
        pfunk.utilities.plot_ci(all_cs_transfer_Norm_l2,colors=[lcolor[ls[1]]], alpha=0.7,zorder=2,levels=[68.0]) 
    
    else:
        pfunk.utilities.plot_ci(all_cs_transfer_Norm, data=d,zorder=1,linewidth=4.0,markersize=10.0,colors=['#AB87FF','#a6cee3'])
    # plt.ylim(min(d.sigma)/2, max(d.sigma)*2)
    if 'twol' in method:
        ymin=min(d.sigma)/10
        ymax=max(d.sigma)*3
    else:
        ymin=min(d.sigma)/2
        ymax=max(d.sigma)*2
    if level in ['8358','8358_1p32','7555','10712','10724','10731','10333','10660']:
        ymin=ymin/5
    if level in ['10712']:
        ymax=ymax*3
    plt.ylim(ymin,ymax)
    plt.xlim(0,30)
    ylabels=[]
    for i in [5e-4,1e-3,2e-3,5e-3,1e-2,2e-2,5e-2,1e-1,2e-1,5e-1]:
        if (i<ymax and i>ymin):
            ylabels.append(i)
    plt.minorticks_on()
    plt.yticks(ylabels,['$'+'{:.0e}'.format(num2tex(i))+'$'  for i in ylabels],fontsize=25)
    plt.xticks(fontsize=25)
    # ax.yaxis.set_major_formatter(LogFormatterMathtext())
    
    plt.tight_layout()
    plt.grid(which='major', axis='both',linestyle='--')
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    plt.tick_params(axis='x',which='both',labelbottom=True)
    plt.tick_params(length=5,width=2)
    if 'twol' in method:
        xshift=0.4
        yshift=0.75
    else:
        xshift=0.3
        yshift=0.17
    if level in ['10660']:
        xshift=0.3
        yshift=0.17
        legendloc='lower right'
    else:
        legendloc='upper right'
    if level=='10724':
        level='10712+10731'
        xshift=xshift+0.1
    if level=='10724' and 'twol' in method:
        level='10712+10731'
        xshift=0.3
        yshift=0.17
    if '8437' in level:
        xshift=0.3
        yshift=0.17
    if 'twol' in method:
        ax.text(0.05+xshift, 0.2+yshift, level[0:level.find('_')]+'keV'  , transform=ax.transAxes, fontsize=25,
                    ha='center',va='top', bbox=props, color='black',weight='bold')
        if level=='8437':
            l12=mpatches.Patch(color='#AB87FF', label='mixed, '+' $\ell='+str(ls[0])+'+'+str(ls[1])+'$' )
            l1 = mpatches.Patch(color=lcolor[ls[0]], label='$J^\pi=1-$, $\ell='+str(ls[0])+'$' )
            l2 = mpatches.Patch(color=lcolor[ls[1]], label='$J^\pi=4+$, $\ell='+str(ls[1])+'$' )
        else:
            l12=mpatches.Patch(color='#AB87FF', label=' $\ell='+str(ls[0])+'+'+str(ls[1])+'$' )
            l1 = mpatches.Patch(color=lcolor[ls[0]], label=' $\ell='+str(ls[0])+'$' )
            l2 = mpatches.Patch(color=lcolor[ls[1]], label=' $\ell='+str(ls[1])+'$' )
        plt.legend(handles=[l12,l1,l2],fontsize=20)
    elif '1p' in method:
        ax.text(xshift, yshift, level[0:level.find('_')]+'keV, '+' $\ell='+str(ls[0])+'$'  , transform=ax.transAxes, fontsize=25,
                ha='center',va='top', bbox=props, color='black',weight='bold')
        plt.legend(handles=[pop_a,pop_b],fontsize=22,loc=legendloc)
    else:
        ax.text(xshift, yshift, level+'keV, '+' $\ell='+str(ls[0])+'$'  , transform=ax.transAxes, fontsize=25,
                ha='center',va='top', bbox=props, color='black',weight='bold')
        
        plt.legend(handles=[pop_a,pop_b],fontsize=22,loc=legendloc)
    plt.xlabel(None)
    plt.ylabel(None)
    plt.errorbar(d.theta, d.sigma, yerr=d.erry,linestyle='None', marker='o',\
                 markersize=0, zorder=4, capsize=4,linewidth=1.5,capthick=1.5,color="black")

    if 'twol' in method:
        plt.savefig(path_plots+'/'+level+'_TR.png')
    else:
        plt.savefig(path_plots+'/'+level+'_TR.png')
    plt.close('all')
    os.remove('new_input')

Loaded level - 9003
Discarded diverged chains: 5610/40000


100%|███████████████████████████████████████| 1000/1000 [03:35<00:00,  4.63it/s]


In [16]:
ymax,ymin

(0.0472885928713222, 0.00240845650483205)

In [ ]:
ANC_results

In [ ]:
Ref.loc[Ref['Level']==levelnum]

In [ ]:
level